# LLM Text Generation

In [1]:
import os
import sys

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../utils'))

from llms import TextGenerationModelFactory

/Users/detraviousjamaribrinkley/Documents/Development/classes/cai_6307_nlp/comp_express_mt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tgmf = TextGenerationModelFactory
llama_31_70b_instruct = tgmf.create_instance('llama-3.1-70b-instruct')
llama_33_70b_instruct = tgmf.create_instance('llama-3.3-70b-instruct')
models = [llama_31_70b_instruct, llama_33_70b_instruct]

In [7]:
llama_31_70b_instruct.model_name

'llama-3.1-70b-instruct'

In [8]:
def generate_data(prompt: str, models: list) -> list:
    model_outputs = {}

    print(f"prompt:\t{prompt}")
    for model in models:

        model_output = model.generate(prompt)
        model_outputs[model.model_name] = model_output
    return model_outputs

In [9]:
prompt = "What is my name?"
generated_data = generate_data(prompt, models)
generated_data

prompt:	What is my name?
	llama-3.1-70b-instruct generated: I don't have any information about your name. I'm a large language model, I don't have personal knowledge about individuals, and our conversation just started, so I haven't received any information about you. If you'd like to share your name, I'd be happy to chat with you!

	llama-3.3-70b-instruct generated: I don't know your name. I'm a large language model, I don't have any information about you, including your name. I'm here to help answer your questions and provide information, but I don't have any personal data about you. Would you like to introduce yourself?



{'llama-3.1-70b-instruct': "I don't have any information about your name. I'm a large language model, I don't have personal knowledge about individuals, and our conversation just started, so I haven't received any information about you. If you'd like to share your name, I'd be happy to chat with you!",
 'llama-3.3-70b-instruct': "I don't know your name. I'm a large language model, I don't have any information about you, including your name. I'm here to help answer your questions and provide information, but I don't have any personal data about you. Would you like to introduce yourself?"}